### Modules


In [ ]:
import numpy as np
import pandas as pd

### Model

In [ ]:
# Intermediate coeff A
def A(Ei, Vij):
  m = len(Ei)
  outerS = 0
  for k in range(m):
    innerS = 0
    for j in range(m):
      innerS += Vij[k][j] * Ei[j]
    outerS += innerS
  return outerS

# Intermediate coeff B
def B(Ei, Vij):
  m = len(Ei)
  outerS = 0
  for k in range(m):
    innerS = 0
    for j in range(m):
      innerS += Vij[k][j] * Ei[j] * Ei[k]
    outerS += innerS
  return outerS

# Intermediate coeff C
def C(Ei, Vij):
  m = len(Ei)
  outerS = 0
  for k in range(m):
    innerS = 0
    for j in range(m):
      innerS += Vij[k][j]
    outerS += innerS
  return outerS

# Intermediate coeff D
def D(a, b, c):
  return b * c - a * a

# Calculation of each Xi
def Xk(k, a, b, c, d, e, Vij, Ei):
  m = len(Ei)
  Lsum = 0
  for j in range(m):
    Lsum += (Vij[k][j] * (c * Ei[j] - a))
  Rsum = 0
  for j in range(m):
    Rsum += (Vij[k][j] * (b -  a * Ei[j]))
  return (e * Lsum + Rsum) / d


def XkR(k, a, b, c, r, e, Vij, Ei):
  m = len(Ei)
  Lsum = 0
  for j in range(m):
    Lsum += (Vij[k][j] * (Ei[j] - r))
  return (e - r) * Lsum / (c*r*r - 2*a*r + b)



def model_risky_only(E, R):
  avg_ret = np.mean(R,axis=1)
  cov_mat = np.cov(R)
  inv_cov = np.linalg.inv(cov_mat)
  m = len(avg_ret)

  # Coefficients
  a = A(avg_ret, inv_cov)
  b = B(avg_ret, inv_cov)
  c = C(avg_ret, inv_cov)
  d = D(a,b,c)

  # Compute each Xi
  outputX = np.zeros(m)
  for k in range(m):
    outputX[k] = Xk(k, a, b, c, d, E, inv_cov, avg_ret)
  return outputX


def model_with_riskless(E, R, RFR):
  avg_ret = np.mean(R,axis=1)
  cov_mat = np.cov(R)
  inv_cov = np.linalg.inv(cov_mat)
  m = len(avg_ret)

  # Coefficients
  a = A(avg_ret, inv_cov)
  b = B(avg_ret, inv_cov)
  c = C(avg_ret, inv_cov)
  # d = D(a,b,c)

  # Compute each Xi
  outputX = np.zeros(m)
  for k in range(m):
    outputX[k] = XkR(k, a, b, c, RFR, E, inv_cov, avg_ret)
  # Calculate risk-free amt
  # outputX[m] = 1 - np.sum(outputX[:-1])
  print('One risk-free asset allowed.')

  return outputX

### Example

In [ ]:
# Inputs
# Desired average annual ROI
E = 0.15
RFR = 0.01

# Historical returns data stored in Drive
rois = '/content/drive/My Drive/MertonModelHistorical/rois.csv' # data from http://www.1stock1.com/1stock1_112.htm (manually cleaned)
syms = '/content/drive/My Drive/MertonModelHistorical/syms.csv' # Data for these stocks: WMT, HD, KO, PG, VZ, TRV, BA, XOM

# Calculations
# Convert to numpy array
rois_pd = pd.read_csv(rois)
R = rois_pd.to_numpy()

# convert to standard list
syms_pd = pd.read_csv(syms)
symbs = list(syms_pd.to_numpy())

# Compute X (distribution)
# X = model_risky_only(E, R)
X = model_with_riskless(E, R, RFR)

# Print results
print('Assumed annual average ROI: '+ str(np.around(E*100,1)) + '%.\n\nMinimium-variance portfolio is:')
for ind, val in enumerate(X):
  print('X('+str(symbs[ind][0])+') =', str(np.around(100 * val)) + '%','(LONG)' if val > 0 else '(SHORT)')
s = np.around(sum(X) * 100)
rfx = 100 - s

print('X(NORISK) = ' + str(rfx) + '%')
print('\nSum of X =',str(s + rfx) + '%')

One risk-free asset allowed.
Assumed annual average ROI: 15.0%.

Minimium-variance portfolio is:
X(WMT) = -5.0% (SHORT)
X(HD) = 23.0% (LONG)
X(KO) = -14.0% (SHORT)
X(PG) = 60.0% (LONG)
X(VZ) = -37.0% (SHORT)
X(TRV) = 23.0% (LONG)
X(BA) = 14.0% (LONG)
X(XOM) = 26.0% (LONG)
X(NORISK) = 11.0%

Sum of X = 100.0%
